In [1]:
from tensorflow.keras.layers import Dense, Input, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import resnet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
batch_size = 64
width, height, channel = 400, 400, 3
no_epochs = 20
verbosity = 1
input_shape = (width, height, channel)

In [3]:
# ResNet50 모델 사용

pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=1000
)

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   rotation_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory('preprocessed_data/train',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory('preprocessed_data/test',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

pretrained_model.trainable=True
# set_trainable=False

# for layer in pretrained_model.layers:
#     if layer.name == 'block7c_project_conv':
#         set_trainable=True
#     if set_trainable:
#         layer.trainable=True
#     else:
#         layer.trainable=False


model=Sequential()
model.add(pretrained_model)
model.add(MaxPooling2D())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[metrics.AUC(name='auc'), 'accuracy'])

Found 1547 images belonging to 2 classes.
Found 387 images belonging to 2 classes.


In [5]:
es_callback = EarlyStopping(monitor='val_auc', mode='max', patience=8,
                                              verbose=1, min_delta=0.0001, restore_best_weights=True)

history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples//batch_size,
                    epochs = no_epochs,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//batch_size,
                    callbacks= [es_callback],
                    verbose=verbosity)

Epoch 1/20
24/24 [==============================] - 55s 2s/step - loss: 1.5825 - auc: 0.6053 - accuracy: 0.7856 - val_loss: 321350.5938 - val_auc: 0.5000 - val_accuracy: 0.8281
Epoch 2/20
24/24 [==============================] - 55s 2s/step - loss: 0.4504 - auc: 0.6708 - accuracy: 0.8247 - val_loss: 6032.1890 - val_auc: 0.5000 - val_accuracy: 0.8307
Epoch 3/20
24/24 [==============================] - 55s 2s/step - loss: 0.4333 - auc: 0.6847 - accuracy: 0.8233 - val_loss: 0.6360 - val_auc: 0.4101 - val_accuracy: 0.8307
Epoch 4/20
24/24 [==============================] - 55s 2s/step - loss: 0.3787 - auc: 0.7900 - accuracy: 0.8260 - val_loss: 0.6871 - val_auc: 0.5000 - val_accuracy: 0.8281
Epoch 5/20
24/24 [==============================] - 55s 2s/step - loss: 0.3652 - auc: 0.8269 - accuracy: 0.8274 - val_loss: 0.6875 - val_auc: 0.5000 - val_accuracy: 0.8307
Epoch 6/20
24/24 [==============================] - 55s 2s/step - loss: 0.3635 - auc: 0.8164 - accuracy: 0.8227 - val_loss: 0.6881 -